In [214]:
import pandas as pd
import sqlite3


Считываем Excel файлы в датафреймы

In [215]:
df = pd.read_excel("Book.xlsx")
df1 = pd.read_excel("Students.xlsx")

Заносим данные в базу SQL **my_base** в таблицу Book из датафрейма(один раз создаем)

In [3]:
# conn = sqlite3.connect('my_base.sqlite')

# table_name = 'Book'

# query = f'Create table if not Exists {table_name} (id_book TEXT,name_book TEXT,genre TEXT,pages TEXT,price TEXT)'
# conn.execute(query)
# df.to_sql(table_name,conn,if_exists='replace',index=False)
# conn.commit()
# conn.close()

In [216]:
conn = sqlite3.connect('my_base.sqlite')
curs=conn.cursor()

curs.execute('SELECT * FROM Book')

rows = list(curs)

for k in range(len(rows)):
    rows[k]=list(rows[k])

In [217]:
from tabulate import tabulate
columns = ["id_book","name_book","genre","pages","price"]
print(tabulate(rows, headers=columns))

  id_book  name_book    genre      pages    price
---------  -----------  -------  -------  -------
        1  name_1       genre_1      100       10
        2  name_2       genre_2      200       20
        3  name_3       genre_3      300       30
        4  name_4       genre_4      500       60
        5  name_5       genre_1      200      100
        6  name_6       genre_4      150       80
        7  name_7       genre_3      200       90
        8  name_8       genre_1      250       40
        9  name_9       genre_2      400       45
       10  name_10      genre_2     1000       60


In [218]:
conn.commit()
conn.close()

Заносим данные в базу SQL **my_base** в таблицу Students из датафрейма(один раз создаем)

In [7]:
# conn = sqlite3.connect('my_base.sqlite')

# table_name = 'Students'

# query = f'Create table if not Exists {table_name} (id_student TEXT,name_student TEXT,id_book TEXT,date TEXT)'
# conn.execute(query)
# df1.to_sql(table_name,conn,if_exists='replace',index=False)

# conn.commit()
# conn.close()

In [219]:
conn = sqlite3.connect('my_base.sqlite')
curs = conn.cursor()
curs.execute('SELECT * FROM Students')

rows = list(curs)

for k in range(len(rows)):
    rows[k]=list(rows[k])

In [220]:
from tabulate import tabulate
columns = ["id_student","name_student","id_book","date"]
print(tabulate(rows, headers=columns))

  id_student  name_student      id_book  date
------------  --------------  ---------  -------------------
           1  name_1                 10  2020-12-20 00:00:00
           2  name_2                  2  2020-12-30 00:00:00
           3  name_3                  3  2020-11-15 00:00:00
           4  name_4                  4  2020-08-31 00:00:00
           5  name_5                  7  2020-07-09 00:00:00
           6  name_6                  6  2020-12-23 00:00:00
           7  name_7                  1  2020-10-30 00:00:00
           8  name_8                  8  2020-09-03 00:00:00
           9  name_9                  9  2020-05-04 00:00:00
          10  name_10                 5  2020-06-20 00:00:00


In [221]:
conn.commit()
conn.close()

In [222]:
conn = sqlite3.connect('my_base.sqlite')
curs = conn.cursor()


1. Вывести самую дорогую книгу


In [223]:
curs.execute(
    '''SELECT name_book, MAX(price)
FROM Book;
'''
)
print(curs.fetchall())

[('name_5', 100)]


In [224]:
curs.execute(
    '''SELECT name_book
    FROM Book
    WHERE price = (SELECT MAX(price) FROM Book);'''
)
print(curs.fetchall())

[('name_5',)]


2. Вывести всех студентов что брали книгу name_3


In [225]:
curs.execute('''SELECT Students.name_student
FROM Students JOIN Book
ON Students.id_book=Book.id_book
WHERE Book.name_book="name_3";''')
print(curs.fetchall())

[('name_3',)]


3. Какое кол-во студентов брали книгу name_2 


In [226]:
curs.execute('''SELECT COUNT(Students.id_student)
FROM Students JOIN Book 
ON Students.id_book = Book.id_book
WHERE Book.name_book="name_2";''')
print(curs.fetchall())

[(1,)]


4. Вывести в алфавитном порядке названия самых дорогих книг в каждом жанре. C условием, что все цены уникальны


In [227]:
curs.execute('''SELECT name_book, MAX(price) 
FROM Book 
GROUP BY genre 
ORDER BY name_book ASC;''')
print(curs.fetchall())

[('name_10', 60), ('name_5', 100), ('name_6', 80), ('name_7', 90)]


In [228]:
curs.execute('''SELECT name_book
FROM Book 
GROUP BY genre 
HAVING price = MAX(price)
ORDER BY name_book ASC;''')
print(curs.fetchall())

[('name_10',), ('name_5',), ('name_6',), ('name_7',)]


5. Вывести имена последних (по дате) трех студентов и книги, которые они брали


In [229]:
curs.execute('''SELECT Students.name_student,Book.name_book
FROM Students JOIN Book
ON Students.id_book=Book.id_book
ORDER BY Students.date DESC
LIMIT 3;'''
)
print(curs.fetchall())

[('name_2', 'name_2'), ('name_6', 'name_6'), ('name_1', 'name_10')]


6. Вывести книги, которые студенты не брали в течении последних 3 месяцев


In [230]:
curs.execute('''SELECT DISTINCT Book.name_book
FROM Students JOIN Book
ON Students.id_book=Book.id_book
WHERE date < date(('now'),'-3 months');
''')
print(curs.fetchall())

[('name_10',), ('name_2',), ('name_3',), ('name_4',), ('name_7',), ('name_6',), ('name_1',), ('name_8',), ('name_9',), ('name_5',)]


In [231]:
curs.execute('''SELECT DISTINCT Book.name_book
FROM Students JOIN Book
ON Students.id_book=Book.id_book
WHERE (julianday('now')-julianday(date)) > 90; 
''')
print(curs.fetchall())

[('name_10',), ('name_2',), ('name_3',), ('name_4',), ('name_7',), ('name_6',), ('name_1',), ('name_8',), ('name_9',), ('name_5',)]


In [232]:
conn.commit()
conn.close()